In [2]:
from sodapy import Socrata
import numpy as np
import pandas as pd

In [25]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("7x5e-2fxh", limit=29000) #feel free to unlimit this if you'd like. 2000 is sufficient for the hackathon (if not doing ML)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df = results_df.replace('Not Available', np.nan)

In [26]:
# drop extraneous columns

# Define the columns to subset based on the categories provided
location_data_columns = ['property_id', 'latitude', 'longitude', 'borough', 'nta']

property_use_details_columns = [
    'primary_property_type',
    'largest_property_use_type',
    'largest_property_use_type_1',
    '_2nd_largest_property_use',
    '_2nd_largest_property_use_1',
    '_3rd_largest_property_use',
    'year_built',
    'construction_status',
    'number_of_buildings',
    'occupancy',
    'metered_areas_energy',
    'metered_areas_water',
    '_3rd_largest_property_use_1',
    'national_median_reference',
    'property_gfa_calculated_1',
    'last_modified_date_property',
    'last_modified_date_electric',
    'last_modified_date_gas_meters',
    'last_modified_date_non',
    'last_modified_date_water',
    'last_modified_date_property_1'
]

energy_use_metrics_columns = [
    'reason_s_for_no_score',
    'energy_star_score',
    'energy_star_certification',
    'energy_star_certification_1',
    'site_eui_kbtu_ft',
    'weather_normalized_site_eui',
    'national_median_site_eui',
    'site_energy_use_kbtu',
    'weather_normalized_site_energy',
    'weather_normalized_site',
    'weather_normalized_site_1',
    'source_eui_kbtu_ft',
    'weather_normalized_source',
    'national_median_source_eui',
    'source_energy_use_kbtu',
    'weather_normalized_source_1',
    'fuel_oil_1_use_kbtu',
    'fuel_oil_2_use_kbtu',
    'fuel_oil_4_use_kbtu',
    'fuel_oil_5_6_use_kbtu',
    'diesel_2_use_kbtu',
    'propane_use_kbtu',
    'district_steam_use_kbtu',
    'district_hot_water_use_kbtu',
    'district_chilled_water_use',
    'natural_gas_use_kbtu',
    'natural_gas_use_therms',
    'weather_normalized_site_2',
    'electricity_use_grid_purchase',
    'electricity_use_grid_purchase_1',
    'weather_normalized_site_3',
    'electricity_use_grid_purchase_2',
    'electricity_use_grid_purchase_3',
    'electricity_use_generated',
    'electricity_use_generated_1',
    'electricity_use_generated_2',
    'electricity_use_generated_3',
    'annual_maximum_demand_kw',
    'annual_maximum_demand_mm',
    'annual_maximum_demand_meter',
    'green_power_onsite_kwh',
    'green_power_offsite_kwh',
    'green_power_onsite_and_offsite',
    'total_ghg_emissions_metric',
    'direct_ghg_emissions_metric',
    'total_ghg_emissions_intensity',
    'direct_ghg_emissions_intensity',
    'indirect_ghg_emissions_metric',
    'net_emissions_metric_tons',
    'indirect_ghg_emissions',
    'national_median_total_ghg',
    'egrid_output_emissions_rate',
    'avoided_emissions_onsite',
    'avoided_emissions_offsite',
    'avoided_emissions_onsite_1',
    'percent_of_recs_retained',
    'percent_of_total_electricity',
    'water_use_all_water_sources',
    'municipally_supplied_potable',
    'municipally_supplied_potable_1',
    'municipally_supplied_potable_2',
    'municipally_supplied_potable_3'
]
data_quality_columns = [
    'estimated_data_flag',
    'estimated_data_flag_natural',
    'estimated_data_flag_fuel',
    'estimated_data_flag_fuel_1',
    'estimated_data_flag_fuel_2',
    'estimated_data_flag_fuel_3',
    'estimated_data_flag_district',
    'estimated_data_flag_1',
    'estimated_values_energy',
    'estimated_values_water',
    'alert_data_center_issue_with',
    'alert_energy_meter_has_less',
    'alert_energy_meter_has_gaps',
    'alert_energy_meter_has',
    'alert_energy_no_meters',
    'alert_energy_meter_has_single',
    'alert_water_meter_has_less',
    'alert_property_has_no_uses'
]

# Combine all columns into a single list
all_subset_columns = location_data_columns + property_use_details_columns + energy_use_metrics_columns + data_quality_columns

# Subset the dataframe based on these columns
df = results_df[all_subset_columns]

In [27]:
df = df.replace('Not Available', np.nan)

In [28]:
# filter for whole metered

df = df.loc[df['metered_areas_water'] == 'Whole Property']
df = df.loc[df['metered_areas_energy'] == 'Whole Property']
df = df.loc[df['construction_status'] == 'Existing']

In [29]:
# Make sure there are no flags on our data
df = df.loc[df['alert_energy_meter_has_less'] == 'Ok']
df = df.loc[df['alert_energy_meter_has_gaps'] == 'Ok']
df = df.loc[df['alert_water_meter_has_less'] == 'Ok']
df = df.loc[df['alert_energy_no_meters'] == 'Ok']

df = df.drop(columns=['metered_areas_water', 'metered_areas_energy', 'construction_status', 'metered_areas_water', 'alert_energy_meter_has_less',
                      'alert_energy_meter_has_gaps', 'alert_water_meter_has_less', 'alert_energy_no_meters'])

df = df.dropna(subset = ['latitude', 'longitude', 'borough', 'nta'])

In [30]:
# Fill NaNs with zeros for the specified columns
fill_zero_columns = [
    'fuel_oil_1_use_kbtu', 'fuel_oil_4_use_kbtu', 'fuel_oil_5_6_use_kbtu',
    'diesel_2_use_kbtu', 'propane_use_kbtu', 'electricity_use_generated',
    'electricity_use_generated_1', 'electricity_use_generated_2', 'electricity_use_generated_3',
    'green_power_onsite_kwh', 'avoided_emissions_onsite', 'percent_of_recs_retained',
    'percent_of_total_electricity'
]
df[fill_zero_columns] = df[fill_zero_columns].fillna(0)

# Drop columns or observations where values are boolean-like
drop_columns = [
    '_3rd_largest_property_use', '_3rd_largest_property_use_1', 'energy_star_certification',
    'estimated_data_flag_district', 'estimated_data_flag_1', 'municipally_supplied_potable',
    'municipally_supplied_potable_3', 'district_steam_use_kbtu', 'district_hot_water_use_kbtu',
    'district_chilled_water_use', 'annual_maximum_demand_kw', 'annual_maximum_demand_mm',
    'annual_maximum_demand_meter', 'municipally_supplied_potable_1', 'municipally_supplied_potable_2'
]

# Drop observations based on specific conditions
df = df[df['estimated_data_flag_fuel'] != "Yes"]
df = df[df['estimated_data_flag_fuel_2'] != "Yes"]
df = df[df['estimated_data_flag_fuel_3'] != "Yes"]
df.drop(columns=drop_columns, inplace=True, errors='ignore')

df = df.drop(columns = [column for column in df.columns if 'alert' in column or 'estimated' in column])

# Drop observations based on specific conditions

In [31]:
dropna_cols = [
    'weather_normalized_site_eui',
    'site_eui_kbtu_ft',
    'national_median_site_eui',
    'site_energy_use_kbtu',
    'weather_normalized_site_2',
    'weather_normalized_site_energy',
    'weather_normalized_site',
    'weather_normalized_site_1',
    'source_eui_kbtu_ft',
    'weather_normalized_source',
    'national_median_source_eui',
    'source_energy_use_kbtu',
    'weather_normalized_source_1',
    'electricity_use_grid_purchase',
    'electricity_use_grid_purchase_1',
    'weather_normalized_site_3',
    'electricity_use_grid_purchase_2',
    'electricity_use_grid_purchase_3',
    'green_power_offsite_kwh',
    'green_power_onsite_and_offsite',
    'total_ghg_emissions_metric',
    'direct_ghg_emissions_metric',
    'total_ghg_emissions_intensity',
    'direct_ghg_emissions_intensity',
    'indirect_ghg_emissions_metric',
    'net_emissions_metric_tons',
    'indirect_ghg_emissions',
    'national_median_total_ghg',
    'egrid_output_emissions_rate',
    'avoided_emissions_offsite',
    'avoided_emissions_onsite_1',
    'water_use_all_water_sources',
    'energy_star_score'
]

df.dropna(subset=dropna_cols, inplace=True)

#Drop specified columns
drop_cols = [
    'nta',
    'reason_s_for_no_score',
    '_2nd_largest_property_use_1',
    '_2nd_largest_property_use',
]

df.drop(columns=drop_cols, inplace=True)

# Fill NaNs with zero for specified columns
fillna_cols = [
    'natural_gas_use_kbtu',
    'natural_gas_use_therms',
    'fuel_oil_2_use_kbtu'
]

df[fillna_cols] = df[fillna_cols].fillna(0)



array(['Multifamily Housing', 'Office', 'Residence Hall/Dormitory',
       'Senior Living Community', 'Retail Store', 'K-12 School',
       'Non-Refrigerated Warehouse', 'Supermarket/Grocery Store', 'Hotel',
       'Medical Office', 'Distribution Center', 'Worship Facility',
       'Financial Office', 'Hospital (General Medical & Surgical)',
       'Mixed Use Property', 'Refrigerated Warehouse', 'Bank Branch',
       'Wastewater Treatment Plant', 'Wholesale Club/Supercenter'],
      dtype=object)

In [32]:
cat_cols = [
    'primary_property_type',
    '_2nd_largest_property_use',
    '_3rd_largest_property_use',
    'metered_areas_energy',
    'metered_areas_water',
    #'national_median_reference',
    'borough',
    #'nta',
    'estimated_data_flag',
    'estimated_data_flag_natural',
    'estimated_data_flag_fuel',
    'estimated_data_flag_fuel_1',
    'estimated_data_flag_fuel_2',
    'estimated_data_flag_fuel_3',
    'estimated_data_flag_district',
    #'estimated_data_flag_1',
    'estimated_values_energy',
    'estimated_values_water',
    'alert_data_center_issue_with',
    'alert_energy_meter_has_less',
    'alert_energy_meter_has_gaps',
    'alert_energy_meter_has',
    'alert_energy_no_meters',
    'alert_energy_meter_has_single',
    'alert_water_meter_has_less',
    'alert_property_has_no_uses'
]

num_cols = [
    'largest_property_use_type_1',
    'year_built',
    'number_of_buildings',
    'occupancy',
    #'_3rd_largest_property_use_1',
    'property_gfa_calculated_1',

    "energy_star_score",

    'site_eui_kbtu_ft',
    'weather_normalized_site_eui',
    #'national_median_site_eui',
    'site_energy_use_kbtu',
    'weather_normalized_site_energy',
    'weather_normalized_site',
    #'weather_normalized_site_1',
    'source_eui_kbtu_ft',
    'weather_normalized_source',
    'national_median_source_eui',
    'source_energy_use_kbtu',
    #'weather_normalized_source_1',
    'fuel_oil_1_use_kbtu',
    #'fuel_oil_2_use_kbtu',
    #'fuel_oil_4_use_kbtu',
    #'fuel_oil_5_6_use_kbtu',
    #'diesel_2_use_kbtu',
    'propane_use_kbtu',
    'district_steam_use_kbtu',
    'district_hot_water_use_kbtu',
    'district_chilled_water_use',
    'natural_gas_use_kbtu',
    'natural_gas_use_therms',
    #'weather_normalized_site_2',
    'electricity_use_grid_purchase',
    'electricity_use_grid_purchase_1',
    #'weather_normalized_site_3',
    #'electricity_use_grid_purchase_2',
    #'electricity_use_grid_purchase_3',
    'electricity_use_generated',
    'electricity_use_generated_1',
    #'electricity_use_generated_2',
    #'electricity_use_generated_3',
    'green_power_onsite_kwh',
    'green_power_offsite_kwh',
    'green_power_onsite_and_offsite',
    #'total_ghg_emissions_metric',
    #'direct_ghg_emissions_metric',
    'total_ghg_emissions_intensity',
    #'direct_ghg_emissions_intensity',
    #'indirect_ghg_emissions_metric',
    #'net_emissions_metric_tons',
    #'indirect_ghg_emissions',
    #'national_median_total_ghg',
    'egrid_output_emissions_rate',
    'avoided_emissions_onsite',
    'avoided_emissions_offsite',
    #'avoided_emissions_onsite_1',
    'percent_of_electricity_that',
    'percent_of_recs_retained',
    'percent_of_total_electricity',
    'water_use_all_water_sources',
    'municipally_supplied_potable',
    #'municipally_supplied_potable_1',
    #'municipally_supplied_potable_2',
    #'municipally_supplied_potable_3'
]

other_cols = [
    'reason_s_for_no_score',
    'energy_star_certification',
    'energy_star_certification_1',
    'annual_maximum_demand_kw',
    'annual_maximum_demand_mm',
    'annual_maximum_demand_meter',
]

date_cols = [
    'last_modified_date_property',
    'last_modified_date_electric',
    'last_modified_date_gas_meters',
    'last_modified_date_non',
    'last_modified_date_water',
    'last_modified_date_property_1'
]

In [11]:
cat_cols = list(set(cat_cols).intersection(df.columns))
num_cols = list(set(num_cols).intersection(df.columns))
other_cols = list(set(other_cols).intersection(df.columns))
date_cols = list(set(date_cols).intersection(df.columns))

def cat2dummy():
    _df = pd.DataFrame()
    for _col in cat_cols:
        _df = pd.concat([
            _df,
            pd.get_dummies(df[[_col]].fillna("nan")).iloc[:, :-1]
        ], axis=1)
    return _df

drop = cat2dummy().sum() / len(df)
assert sum(drop == 1) == 0

for _col in num_cols:
    try:
        df[_col].astype(np.float64)
    except:
        print(_col)
        
X_num = df[num_cols]
X_cat = cat2dummy()


for col in X_num.columns:
  try:
    X_num[col] = X_num[col].astype(np.float64)
  except:
    X_num.drop(columns=[col], inplace=True)
df = pd.concat([X_num, X_cat], axis=1)

natural_gas_use_therms
electricity_use_grid_purchase
natural_gas_use_kbtu
electricity_use_grid_purchase_1


/var/folders/sv/b2zydtkj4_14gky1s46kj64c0000gn/T/ipykernel_7479/2289750009.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_num[col] = X_num[col].astype(np.float64)
/var/folders/sv/b2zydtkj4_14gky1s46kj64c0000gn/T/ipykernel_7479/2289750009.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_num[col] = X_num[col].astype(np.float64)
/var/folders/sv/b2zydtkj4_14gky1s46kj64c0000gn/T/ipykernel_7479/2289750009.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra